In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import tensorflow_decision_forests as tfdf
print(f"Found TF-DF {tfdf.__version__}")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
serving_df = pd.read_csv("/kaggle/input/titanic/test.csv")
serving_df.head()
serving_df

In [ ]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
train_df.head()

Preparar base de datos:
Aplicar las siguientes tranformacion en la base de datos

1. Tokenizazr los nombres."Seather, Mr. Simon Sivertsen" will become ["Seather", "Mr.", "Simon", "Sivertsen"].
2. Extraer los prefijos en los tickets. Ejemplo "SOTON/O.Q. 3101262" VA A SER "SOTON/0.Q." y 31012262

In [ ]:
def preprocess(df):
    df = df.copy()
    
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    
    def ticket_number(x):
        return x.split(" ")[-1]
    
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
    
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)
    return df

preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)

preprocessed_train_df.head(5)

Mantengamos la lista de las características de entrada del modelo. En particular, no queremos entrenar nuestro modelo con las características "PassengerId" y "Ticket".

In [ ]:
input_features = list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")
#input_features.remove("Ticket_number")

print(f"Input features: {input_features}")

Convertir el dataset Pandas en dataset TensorFlow 

In [ ]:
def tokenize_names(features, labels=None):
    """Divida los nombres en tokens. TF-DF puede consumir tokens de texto de forma nativa."""
    features["Name"] =  tf.strings.split(features["Name"])
    return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df,label="Survived").map(tokenize_names)
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)

Entrenar el modelo con parametros por defecto

En primer lugar, vamos a entrenar un modelo GradientBoostedTreesModel con los parámetros por defecto.

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, #only use the features in "features"
    random_seed=1234,
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Losss:{self_evaluation.loss}")

El resultado muestra que el modelo ha sido entrenado y evaluado en el conjunto de entrenamiento. La precisión del modelo en el conjunto de entrenamiento es de 0.826, lo que es un buen punto de partida.

## Entrenar el modelo con parámetros por defecto mejorados
Ahora utilizarás algunos parámetros específicos al crear el modelo GBT

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    
    #num_trees=2000,
    
    # Only for GBT.
    # A bit slower, but great to understand the model.
    # compute_permutation_variable_importance=True,
    
    # Change the default hyper-parameters
    # hyperparameter_template="benchmark_rank1@v1",
    
    #num_trees=1000,
    #tuner=tuner
    
    min_examples=1,
    categorical_algorithm="RANDOM",
    #max_depth=4,
    shrinkage=0.05,
    #num_candidate_attributes_ratio=0.2,
    split_axis="SPARSE_OBLIQUE",
    sparse_oblique_normalization="MIN_MAX",
    sparse_oblique_num_projections_exponent=2.0,
    num_trees=2000,
    #validation_ratio=0.0,
    random_seed=1234

)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

El output muestra que el modelo ha sido cargado desde el directorio especificado en el mensaje [INFO], y evaluado en el conjunto de entrenamiento. La precisión del modelo en el conjunto de entrenamiento es 0.7608, lo que puede ser un poco inferior a lo que se obtuvo previamente en el modelo básico.

In [ ]:
model.summary()

El modelo presentado es un Gradient Boosted Trees Model para la tarea de clasificación, con una única capa y 11 características de entrada, como la edad, la tarifa y el sexo. La precisión del modelo se mejoró con el tiempo a medida que se agregaron más iteraciones, y la precisión final fue del 93.87% para el conjunto de entrenamiento y del 77.17% para el conjunto de validación. La importancia de las variables se muestra en diferentes categorías, como INV_MEAN_MIN_DEPTH, NUM_AS_ROOT, NUM_NODES y SUM_SCORE. El modelo tiene un total de 33 árboles, con un promedio de 55 nodos por árbol. El tipo de condición en los nodos es principalmente oblicua, contiene condiciones y contiene condición de mapa de bits.

## Hacer predicciones

In [ ]:
def prediction_to_kaggle_format(model, threshold=0.5):
    proba_survive = model.predict(serving_ds, verbose=0)[:,0]
    return pd.DataFrame({
        "PassengerId": serving_df["PassengerId"],
        "Survived": (proba_survive >= threshold).astype(int)
    })

def make_submission(kaggle_predictions):
    path="/kaggle/working/submission.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")
    
kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)
!head /kaggle/working/submission.csv

El código presentado es una función que convierte las predicciones del modelo en un formato que es compatible con la competencia de Kaggle y luego las exporta como un archivo CSV en la ubicación de trabajo actual en Kaggle.

La función prediction_to_kaggle_format toma el modelo entrenado y utiliza el conjunto de datos de prueba serving_ds para hacer predicciones. A continuación, la función crea un DataFrame de Pandas que contiene las columnas "PassengerId" y "Survived", donde "PassengerId" es el identificador del pasajero y "Survived" es la predicción binaria de si ese pasajero sobrevivió o no. La predicción se realiza mediante el umbral (threshold) de probabilidad especificado como parámetro en la función. Si la probabilidad de supervivencia del pasajero es mayor o igual que el umbral, entonces la predicción es 1, de lo contrario, es 0.

La función make_submission toma el DataFrame creado por prediction_to_kaggle_format y lo guarda como un archivo CSV en la ubicación especificada en el parámetro path. Finalmente, el código muestra las primeras líneas del archivo CSV creado utilizando el comando !head.

En general, el objetivo de este código es hacer predicciones sobre el conjunto de datos de prueba y crear un archivo de sumisión que pueda ser cargado en la plataforma Kaggle para evaluar el rendimiento del modelo en una competencia de aprendizaje automático.

## Entrenamiento de un modelo con ajuste de hiperparámetros
La sintonización de hiperparámetros se activa especificando el argumento constructor del sintonizador del modelo. El objeto tuner contiene toda la configuración del tuner (espacio de búsqueda, optimizador, prueba y objetivo).

In [ ]:
tuner = tfdf.tuner.RandomSearch(num_trials=1000)
tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 8])

global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 2256])

#tuner.choice("use_hessian_gain", [True, False])
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])


tuner.choice("split_axis", ["AXIS_ALIGNED"])
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice("sparse_oblique_normalization",
                    ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])

# Tune the model. Notice the `tuner=tuner`.
tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds, verbose=0)

tuned_self_evaluation = tuned_model.make_inspector().evaluation()
print(f"Accuracy: {tuned_self_evaluation.accuracy} Loss:{tuned_self_evaluation.loss}")

En esta sección se muestra cómo utilizar la optimización de hiperparámetros para mejorar el rendimiento del modelo. La optimización de hiperparámetros se realiza especificando el argumento constructor tuner del modelo. El objeto tuner contiene toda la configuración del optimizador (espacio de búsqueda, optimizador, ensayos y objetivo).

Se definen diferentes espacios de búsqueda para los hiperparámetros a través de la función tuner.choice(), en la que se especifican las opciones para cada parámetro. Luego se entrena el modelo utilizando el optimizador tuner y se muestra la precisión obtenida.

El objetivo de la optimización de hiperparámetros es encontrar la mejor combinación de valores de hiperparámetros para mejorar la precisión del modelo. La optimización de hiperparámetros puede ser automatizada y es muy útil cuando se trabaja con grandes conjuntos de datos y se necesita mejorar la precisión del modelo.

Para más información se recomienda seguir el tutorial de optimización de hiperparámetros automatizada.

## Creando un conjunto (ensemble)
Aquí crearás 100 modelos con diferentes semillas y combinarás sus resultados.

Este enfoque elimina un poco los aspectos aleatorios relacionados con la creación de modelos de aprendizaje automático.

En la creación de GBT se utiliza el parámetro "honest". Utilizará diferentes ejemplos de entrenamiento para inferir la estructura y los valores de las hojas. Esta técnica de regularización intercambia ejemplos por estimaciones de sesgo.

In [ ]:
predictions = None
num_predictions = 0

for i in range(100):
    print(f"i:{i}")
    # Possible models: GradientBoostedTreesModel or RandomForestModel
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose=0, # Very few logs
        features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
        exclude_non_specified_features=True, # Only use the features in "features"

        #min_examples=1,
        #categorical_algorithm="RANDOM",
        ##max_depth=4,
        #shrinkage=0.05,
        ##num_candidate_attributes_ratio=0.2,
        #split_axis="SPARSE_OBLIQUE",
        #sparse_oblique_normalization="MIN_MAX",
        #sparse_oblique_num_projections_exponent=2.0,
        #num_trees=2000,
        ##validation_ratio=0.0,
        random_seed=i,
        honest=True,
    )
    model.fit(train_ds)
    
    sub_predictions = model.predict(serving_ds, verbose=0)[:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num_predictions += 1

predictions/=num_predictions

kaggle_predictions = pd.DataFrame({
        "PassengerId": serving_df["PassengerId"],
        "Survived": (predictions >= 0.5).astype(int)
    })

make_submission(kaggle_predictions)